In [1]:
# necessary imports
import torch
import os
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc
import torch.optim as optim
from torchvision import transforms, datasets, models
from torch.utils.data import DataLoader
from openood.evaluation_api import Evaluator
from openood.networks import ResNet18_32x32

In [2]:
net = ResNet18_32x32(num_classes=10)
net.load_state_dict(
    torch.load('./results/cifar10_resnet18_32x32_base_e100_lr0.1_default/s0/best.ckpt')
)
net.cuda()
net.eval()

C:\Users\Zhao\AppData\Local\Temp\ipykernel_19704\1368810468.py:3: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  torch.load('./results/cifar10_resnet18_32x32_base_e100_lr0.1_

ResNet18_32x32(
  (conv1): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (shortcut): Sequential()
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1,

In [3]:
transform_cifar10 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.4914, 0.4822, 0.4465], [0.2470, 0.2435, 0.2616]),
])

In [4]:
transform_cifar100 = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize([0.5071, 0.4867, 0.4408], [0.2675, 0.2565, 0.2761]),
])

In [6]:
train_dir10 = './data/images_classic/cifar10/cifar10/train'
test_dir10 = './data/images_classic/cifar10/cifar10/test'
train_dir100 = './data/images_classic/cifar100/cifar100/train'
test_dir100 = './data/images_classic/cifar100/cifar100/test'

In [7]:
datasets10 = {
    'train': datasets.ImageFolder(train_dir10, transform=transform_cifar10),
    'test': datasets.ImageFolder(test_dir10, transform=transform_cifar10)
}
datasets100 = {
    'train': datasets.ImageFolder(train_dir100, transform=transform_cifar100),
    'test': datasets.ImageFolder(test_dir100, transform=transform_cifar100)
}

In [8]:
dataloaders10 = {
    'train': DataLoader(datasets10['train'], batch_size=64, shuffle=True, num_workers=4),
    'test': DataLoader(datasets10['test'], batch_size=64, shuffle=False, num_workers=4)
}
dataloaders100 = {
    'train': DataLoader(datasets100['train'], batch_size=64, shuffle=True, num_workers=4),
    'test': DataLoader(datasets100['test'], batch_size=64, shuffle=False, num_workers=4)
}

# using training dataset for feature extraction

In [9]:
def extract_features(model, dataloader):
    features_list = []
    labels_list = []

    model.eval()  # Set the model to evaluation mode
    with torch.no_grad():
        for data in dataloader:
            inputs, labels = data[0].cuda(), data[1].cuda()

            # Extract features from the model (after avgpool)
            x = model.conv1(inputs)
            x = model.bn1(x)
            x = model.layer1(x)
            x = model.layer2(x)
            x = model.layer3(x)
            x = model.layer4(x)
            x = model.avgpool(x)  # Apply avgpool to reduce dimensions
            
            # Flatten the features to make them 2D
            features = torch.flatten(x, 1)  # Now the features are (batch_size, num_features)

            # Append to the list
            features_list.append(features.cpu().numpy())
            labels_list.append(labels.cpu().numpy())
    
    return np.concatenate(features_list), np.concatenate(labels_list)


In [10]:
# Then extract features
train_features, train_labels = extract_features(net, dataloaders10["train"])

In [11]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import roc_curve
import numpy as np

In [12]:
# Train a KNN model using CIFAR-10 training features
knn = KNeighborsClassifier(n_neighbors=50)
knn.fit(train_features, train_labels)

KNeighborsClassifier(n_neighbors=50)

In [13]:
ID_loader = torch.utils.data.DataLoader(datasets10['test'], batch_size=64, shuffle=False)
OOD_loader = torch.utils.data.DataLoader(datasets100['test'], batch_size=64, shuffle=False)

# KNN scores

In [14]:
def detect_ood(knn, features):
    """
    Detect OOD samples based on the KNN distances.
    
    Args:
        knn (KNeighborsClassifier): Trained KNN model.
        features (np.array): Extracted features from the model.
    
    Returns:
        ood_scores (list): List of mean distances to nearest neighbors.
    """
    # Use KNN to compute distances to the nearest neighbors
    distances, _ = knn.kneighbors(features)

    # Calculate the mean distance to the nearest neighbors
    mean_distances = np.mean(distances, axis=1)
    
    return mean_distances

In [15]:
def calculate_fpr_at_95_tpr(id_scores, ood_scores):
    """
    Calculate FPR@95TPR.
    
    Args:
        id_scores (list): Distance scores for in-distribution data.
        ood_scores (list): Distance scores for out-of-distribution data.
    
    Returns:
        fpr_at_95_tpr (float): False Positive Rate at 95% True Positive Rate.
    """
    # Create labels for ID (1) and OOD (0)
    labels = np.concatenate([np.ones(len(id_scores)), np.zeros(len(ood_scores))])

    # Concatenate the scores (distances)
    scores = np.concatenate([id_scores, ood_scores])

    # Compute the false positive rate (FPR), true positive rate (TPR), and thresholds
    fpr, tpr, thresholds = roc_curve(labels, -scores)  # Negative scores because smaller distances = higher confidence

    # Find the threshold where TPR is closest to 95%
    index_95_tpr = np.where(tpr >= 0.95)[0][0]

    # FPR at the threshold where TPR is 95%
    fpr_at_95_tpr = fpr[index_95_tpr]

    return fpr_at_95_tpr

In [16]:
# Step 3: Extract features from the test CIFAR-10 (in-distribution) and CIFAR-100 (OOD) datasets
id_features, _ = extract_features(net, ID_loader)  # In-distribution (first 4 classes)
ood_features, _ = extract_features(net, OOD_loader)  # OOD (remaining 6 classes)

# Step 4: Use KNN to detect OOD based on distances
id_scores = detect_ood(knn, id_features)
ood_scores = detect_ood(knn, ood_features)

# Step 5: Calculate FPR@95TPR
fpr_at_95 = calculate_fpr_at_95_tpr(id_scores, ood_scores)
print(f"FPR@95TPR: {fpr_at_95:.4f}")

FPR@95TPR: 0.6243


In [17]:

# Step 2: Train KNN on the in-distribution features
knn = KNeighborsClassifier(n_neighbors=10)
knn.fit(train_features, train_labels)
# Step 3: Extract features from the test CIFAR-10 (in-distribution) and CIFAR-100 (OOD) datasets
id_features, _ = extract_features(net, ID_loader)  # In-distribution (first 4 classes)
ood_features, _ = extract_features(net, OOD_loader)  # OOD (remaining 6 classes)

# Step 4: Use KNN to detect OOD based on distances
id_scores = detect_ood(knn, id_features)
ood_scores = detect_ood(knn, ood_features)

# Step 5: Calculate FPR@95TPR
fpr_at_95 = calculate_fpr_at_95_tpr(id_scores, ood_scores)
print(f"FPR@95TPR: {fpr_at_95:.4f}")

FPR@95TPR: 0.6163


In [18]:

# Step 2: Train KNN on the in-distribution features
knn = KNeighborsClassifier(n_neighbors=100)
knn.fit(train_features, train_labels)
# Step 3: Extract features from the test CIFAR-10 (in-distribution) and CIFAR-100 (OOD) datasets
id_features, _ = extract_features(net, ID_loader)  # In-distribution (first 4 classes)
ood_features, _ = extract_features(net, OOD_loader)  # OOD (remaining 6 classes)

# Step 4: Use KNN to detect OOD based on distances
id_scores = detect_ood(knn, id_features)
ood_scores = detect_ood(knn, ood_features)

# Step 5: Calculate FPR@95TPR
fpr_at_95 = calculate_fpr_at_95_tpr(id_scores, ood_scores)
print(f"FPR@95TPR: {fpr_at_95:.4f}")

FPR@95TPR: 0.6302
